# The goal of this project is to create Systematic - Automatic approach of dealing with ML prototyping

## First we always start with Reading Data, Visualizing Data, Cleaning Data

In [ ]:
import pandas as pd
import datetime
import numpy as np

from time import gmtime
from time import strftime

import matplotlib.pyplot as plt

dataFrame = pd.read_csv('Q3-data.csv') 
print(dataFrame.head())

# Before anything starts we need to formulate how to set up our data
If this is a supervised learning problem we will need a clear input output data, so we will have to clean this and creat a clear input output connection

In this particular problem we can do that using the TV column, we translate this information of TV current cunsamtion into a binarry groundtruth classification between open and closed TV

In [ ]:
allBackets = []
todayBackets = []
housePr = -1
for i in range(len(dataFrame.TV)):
    
    houseNow = int(dataFrame.House[i])-1
    if (housePr!=houseNow):
        if (housePr!=-1):
            allBackets.append(todayBackets)
        todayBackets = []
        todayBackets.append([dataFrame.TV[i],dataFrame.Agg[i],0])
        housePr = houseNow
    else:
        todayBackets.append([dataFrame.TV[i],dataFrame.Agg[i],0])
        
        
allBackets.append(todayBackets)

# print(allBackets[0]) # information of House 0
print(allBackets[0][20]) # information of House 0 hour 20
print(allBackets[0][20][0]) # information of House 0 hour 20 TV current 0 -> 0 = TV Current / 1 = All House Current / 2 = Ground Truth Open Close



MaxMinHouse = np.zeros((2,len(allBackets)))-1 # 0 -> max # 1 -> min
MaxMinHouse[1,:] = float("Inf")
for houseN in range(len(allBackets)):
    for i in range(len(allBackets[houseN])):
        if MaxMinHouse[0,houseN]<allBackets[houseN][i][0]:
            MaxMinHouse[0,houseN] = allBackets[houseN][i][0]
        if MaxMinHouse[1,houseN]>allBackets[houseN][i][0]:
            MaxMinHouse[1,houseN] = allBackets[houseN][i][0]
            
            
print("max,min = ",MaxMinHouse[0,0],MaxMinHouse[1,0])
print("max,min = ",MaxMinHouse[0,1],MaxMinHouse[1,1])
print("max,min = ",MaxMinHouse[0,2],MaxMinHouse[1,2])
       

# Visualization of the Data and Changing info to GroundTruth 

In [ ]:
from functionsModelTrain.cleaningData import balanced_hist_thresholding


for houseN in range(len(allBackets)): 
    showNow = np.zeros((int(MaxMinHouse[0,houseN])+1,1))
    printSignal = []
    for i in range(len(allBackets[houseN])):
        showNow[int(allBackets[houseN][i][0])] = showNow[int(allBackets[houseN][i][0])] + 1
        printSignal.append(allBackets[houseN][i][0])

    print("House = ",houseN)
    
    print("Histogram")
    plt.plot(showNow)
    plt.show()

    thresh_value = balanced_hist_thresholding(showNow)
    print("threshold = ",thresh_value)
    
    
    print("Raw Signal")
    plt.plot(printSignal)
    plt.show()
    
    showThreshold = []
    for i in range(len(allBackets[houseN])):
        if (int(allBackets[houseN][i][0])>thresh_value):
            allBackets[houseN][i][2] = 1 
        else:
            allBackets[houseN][i][2] = 0 
            
        showThreshold.append(allBackets[houseN][i][2])
      
    print("Signal Translated to binary")
    plt.plot(showThreshold)
    plt.show()
    

# Different visualization using Backets

In [ ]:

maxTV = max(dataFrame.Agg)
minTV = min(dataFrame.Agg)
print(maxTV,minTV)

maxTV = 130

backetsAvailable = 50 # This is the numbers of areas that we will have 
areaPerBacket = (maxTV-minTV)/backetsAvailable

todayBackets = np.zeros((backetsAvailable+1,3))
for i in range(len(dataFrame.TV)):
    if (dataFrame.Agg[i]<130):
        backetNow = int(dataFrame.Agg[i]/areaPerBacket)
        todayBackets[backetNow,int(dataFrame.House[i])-1] = todayBackets[backetNow,int(dataFrame.House[i])-1] + 1
    

plt.plot(todayBackets[:,0])
plt.show()   
plt.plot(todayBackets[:,1])
plt.show()   
plt.plot(todayBackets[:,2])
plt.show()   

In [ ]:
def visualise_data (allBackets):
    print "1s in blue, 0s in orange"
    for i in range(3):
        print
        print "House ",i
        ones = [v for v in allBackets[i] if v[2] == 1]
        ones = np.array(ones)
        zeros = [v for v in allBackets[i] if v[2] == 0]
        zeros = np.array(zeros)

        plt.plot(ones[:,1])
        plt.plot(zeros[:,1])
        plt.show()
        print "ones: ", len(ones)
        print "min",np.min(ones[:,1]), "max", np.max(ones[:,1]), "mean", np.mean(ones[:,1])
        print
        print "zeros: ",len(zeros)
        print "min",np.min(zeros[:,1]), "max", np.max(zeros[:,1]), "mean", np.mean(zeros[:,1])
        print "------------------------------------------"
        
visualise_data(allBackets)


# Find the outliers 

In [ ]:
from functionsModelTrain.cleaningData import find_outliers

house0 = allBackets[0]
house1 = allBackets[1]
house2 = allBackets[2]

data = []
data.extend(house0) 
data.extend(house1) 
data.extend(house2)

data = np.array(data)
data = data[:,1] 
print len(house0), len(house1), len(house2)
print len(data)

lower_bound, upper_bound = find_outliers (data)


# UNDERSAMPLING (REMOVE OUTLIERS FROM DATA) & VISUALISATION

In [ ]:
def undersampling (allBackets):

    underSampl1 = [v for v in allBackets[0] if v[1] < upper_bound]
    underSampl2 = [v for v in allBackets[1] if v[1] < upper_bound]
    underSampl3 = [v for v in allBackets[2] if v[1] < upper_bound]

    return underSampl1,underSampl2,underSampl3


underSampl1,underSampl2,underSampl3 = undersampling(allBackets)

allBackets2=[]
allBackets2.append(underSampl1)
print (len(allBackets2[0]))
allBackets2.append(underSampl2)
print (len(allBackets2[1]))
allBackets2.append(underSampl3)
print (len(allBackets2[2]))

print (allBackets2[0][0])


In [ ]:
visualise_data(allBackets2)

# Separate into Train and Test Data with multiple different combinations

In [ ]:
from functionsModelTrain.trainTest import train_test_sets


#ALL HOUSES ORIGINAL
train_data_org , train_labels_org, test_data_org , test_labels_org = train_test_sets(allBackets, 0.7, House0=1, Outliers_train=1, Outliers_test=1)
#ALL HOUSES :TRAIN_DATA -> NO OUTLIERS
train_data , train_labels,test_data , test_labels = train_test_sets(allBackets, 0.7, House0=1, Outliers_train=0, Outliers_test=1)
#ALL HOUSES : TRAIN_DATA & TEST DATA -> NO OUTLIERS"
train_data_noOUT , train_labels_noOUT , test_data_noOUT  , test_labels_noOUT = train_test_sets(allBackets, 0.7, House0=1, Outliers_train=0, Outliers_test=0)


#ORIGINAL NO HOUSE 0
train_data_noHouse0_org , train_labels_noHouse0_org, test_data_noHouse0_org  , test_labels_noHouse0_org = train_test_sets(allBackets, 0.7, House0=0, Outliers_train=1, Outliers_test=1)
#NO HOUSE 0 : TRAIN_DATA -> NO OUTLIERS
train_data_noHouse0 , train_labels_noHouse0 ,test_data_noHouse0  , test_labels_noHouse0= train_test_sets(allBackets, 0.7, House0=0, Outliers_train=0, Outliers_test=1)
#"NO HOUSE 0 : TRAIN_DATA & TEST DATA -> NO OUTLIERS"
train_data_noHouse0_noOUT , train_labels_noHouse0_noOUT , test_data_noHouse0_noOUT  , test_labels_noHouse0_noOUT = train_test_sets(allBackets, 0.7, House0=0, Outliers_train=0, Outliers_test=0)




# Test different models in your trainin data

In [ ]:
from functionsModelTrain.models import logisticRegression_classifier

print "ALL HOUSES ORIGINAL"
modelLogReg = logisticRegression_classifier(train_data_org, train_labels_org)


# TEST model and accuracy metrics

In [ ]:
from functionsModelTrain.testModelAccur import accuracyScoreModel,confussionMatr,classification_reportRes

print(accuracyScoreModel(modelLogReg,test_data,test_labels))
print(confussionMatr(modelLogReg,test_data,test_labels))
print(classification_reportRes(modelLogReg,test_data,test_labels))

In [ ]:
from functionsModelTrain.models import RandomForest_Classifier


modelRFC = RandomForest_Classifier(train_data_org, train_labels_org,100)


In [ ]:
from functionsModelTrain.testModelAccur import accuracyScoreModel,confussionMatr,classification_reportRes

print(accuracyScoreModel(modelRFC,test_data,test_labels))
print(confussionMatr(modelRFC,test_data,test_labels))
print(classification_reportRes(modelRFC,test_data,test_labels))

In [ ]:
from functionsModelTrain.models import SVM_classifier


modelSVM = SVM_classifier(train_data_org, train_labels_org,'rbf',30,'auto')
# modelSVM = SVM_classifier(train_data_org, train_labels_org)


In [ ]:
from functionsModelTrain.testModelAccur import accuracyScoreModel,confussionMatr,classification_reportRes

print(accuracyScoreModel(modelSVM,test_data,test_labels))
print(confussionMatr(modelSVM,test_data,test_labels))
print(classification_reportRes(modelSVM,test_data,test_labels))

In [ ]:
from functionsModelTrain.models import NeuralNetwork_Classifier


# modelNN = NeuralNetwork_Classifier(train_data_org, train_labels_org,[5,12,18,12,5],500)
modelNN = NeuralNetwork_Classifier(train_data_org, train_labels_org,[11,18,11],1500)


In [ ]:
from functionsModelTrain.testModelAccur import accuracyScoreModel,confussionMatr,classification_reportRes

print(accuracyScoreModel(modelNN,test_data,test_labels))
print(confussionMatr(modelNN,test_data,test_labels))
print(classification_reportRes(modelNN,test_data,test_labels))

# GridSearch for HyperParameter Tunning

In [ ]:
from sklearn.neural_network import MLPClassifier
from functionsModelTrain.hyperParameterSearch import hyperParameterSearch

# hyperParameters = {
#     'hidden_layer_sizes':[[10,10],[5,10,5]],
#     'max_iter':[500,1000,2000]
# }
hyperParameters = {
    'hidden_layer_sizes':[[10,10],[5,10,5],[5,10,20,10,5],[5,5,5]],
    'max_iter':[500,1000,2000,5000]
}

train_data_orgNow = train_data_org.reshape(-1,1)

mlpc = hyperParameterSearch(MLPClassifier(),hyperParameters,"randG",10,train_data_orgNow, train_labels_org)

In [ ]:
df = pd.DataFrame(mlpc.cv_results_)
print(df[['param_hidden_layer_sizes','param_max_iter','mean_test_score']])

print(mlpc.best_score_)
print(mlpc.best_params_)


# Use the Best Model

In [ ]:
from functionsModelTrain.models import NeuralNetwork_Classifier


hidden_layerN = mlpc.best_params_["hidden_layer_sizes"]
max_iterN = mlpc.best_params_["max_iter"]
modelNN = NeuralNetwork_Classifier(train_data_org, train_labels_org,hidden_layerN,max_iterN)


In [ ]:
from functionsModelTrain.testModelAccur import accuracyScoreModel,confussionMatr,classification_reportRes

print(accuracyScoreModel(modelNN,test_data,test_labels))
print(confussionMatr(modelNN,test_data,test_labels))
print(classification_reportRes(modelNN,test_data,test_labels))

# Evaluate Classifiers

Here we will calcualte accuracy score, but we always have to compare this with nul acuracy which is what is the persentage if we just calucate everything to be equal to 0

In [ ]:
from sklearn import metrics

test_data = test_data.reshape(-1,1)

predictions = modelNN.predict(test_data)

print(metrics.accuracy_score(test_labels,predictions))

In [ ]:
print(1 - test_labels.mean()) # This is the minimum prediction if the system is completly random

# Looking the actual predictions 

In [ ]:
stepN = 15
numberLines = 3

startPoint = 0


for i in range(len(test_labels)/stepN):
    print(i)
    print("True: ",test_labels[i:i+stepN])
    print("Pred: ",predictions[i:i+stepN])
    
    ConfMatrix = []
    for kt in range(stepN):
        if (test_labels[i+kt]==1 and predictions[i+kt]==1):
            ConfMatrix.append("TP")
        elif (test_labels[i+kt]==0 and predictions[i+kt]==0):
            ConfMatrix.append("TN")
        elif (test_labels[i+kt]==1 and predictions[i+kt]==0):
            ConfMatrix.append("FN")
        elif (test_labels[i+kt]==0 and predictions[i+kt]==1):
            ConfMatrix.append("FP")
            
    print("ConM:'array",ConfMatrix)
            
    if (i>=numberLines):
        break
    
    
    startPoint = startPoint + stepN

In [ ]:
from functionsModelTrain.testModelAccur import VisualizePredTestResults


VisualizePredTestResults(modelNN,test_data,test_labels,12,5)